In [16]:
import torchvision.datasets as dset
from torch.utils.data import Dataset, RandomSampler
from torchvision.transforms import v2
import torch
from torch.utils.data import DataLoader
import glob
from PIL import Image
import os
import scipy
import torchfile

In [13]:
mat = scipy.io.loadmat("/Users/buckhousefamily/torch_datasets/flowers-full/flowers-102/imagelabels.mat")

In [14]:
mat

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNX86, Created on: Thu Feb 19 15:43:33 2009',
 '__version__': '1.0',
 '__globals__': [],
 'labels': array([[77, 77, 77, ..., 62, 62, 62]], dtype=uint8)}

In [ ]:
img_size = 64
transforms = v2.Compose([
    #v2.PILToTensor(),
    v2.ToImage(), 
    v2.ToDtype(torch.float32, scale=True),
    v2.Resize(img_size),
    v2.CenterCrop(size=(img_size, img_size))
    

])

In [3]:
flowers = dset.Flowers102(root=os.path.expanduser("~/torch_datasets/flowers-full"), download=True)

100%|██████████| 344862509/344862509 [00:17<00:00, 19251870.54it/s]


Extracting /Users/buckhousefamily/torch_datasets/flowers-full/flowers-102/102flowers.tgz to /Users/buckhousefamily/torch_datasets/flowers-full/flowers-102


100%|██████████| 502/502 [00:00<00:00, 273127.59it/s]


100%|██████████| 14989/14989 [00:00<00:00, 10591041.55it/s]


In [4]:
# https://github.com/TDeVries/cub2011_dataset/blob/master/cub2011.py

import pandas as pd
from torchvision.datasets.folder import default_loader
from torchvision.datasets.utils import download_url
from torch.utils.data import Dataset
class Cub2011(Dataset):
    base_folder = 'CUB_200_2011/images'
    url = 'https://data.caltech.edu/records/65de6-vp158/files/CUB_200_2011.tgz?download=1'
    filename = 'CUB_200_2011.tgz'
    tgz_md5 = '97eceeb196236b17998738112f37df78'

    def __init__(self, root, train=True, transform=None, loader=default_loader, download=True):
        self.root = os.path.expanduser(root)
        self.transform = transform
        self.loader = default_loader
        self.train = train

        if download:
            self._download()

        if not self._check_integrity():
            raise RuntimeError('Dataset not found or corrupted.' +
                               ' You can use download=True to download it')

    def _load_metadata(self):
        images = pd.read_csv(os.path.join(self.root, 'CUB_200_2011', 'images.txt'), sep=' ',
                             names=['img_id', 'filepath'])
        image_class_labels = pd.read_csv(os.path.join(self.root, 'CUB_200_2011', 'image_class_labels.txt'),
                                         sep=' ', names=['img_id', 'target'])
        train_test_split = pd.read_csv(os.path.join(self.root, 'CUB_200_2011', 'train_test_split.txt'),
                                       sep=' ', names=['img_id', 'is_training_img'])

        data = images.merge(image_class_labels, on='img_id')
        self.data = data.merge(train_test_split, on='img_id')

        if self.train:
            self.data = self.data[self.data.is_training_img == 1]
        else:
            self.data = self.data[self.data.is_training_img == 0]

    def _check_integrity(self):
        try:
            self._load_metadata()
        except Exception:
            return False

        for index, row in self.data.iterrows():
            filepath = os.path.join(self.root, self.base_folder, row.filepath)
            if not os.path.isfile(filepath):
                print(filepath)
                return False
        return True

    def _download(self):
        import tarfile

        if self._check_integrity():
            print('Files already downloaded and verified')
            return

        download_url(self.url, self.root, self.filename, self.tgz_md5)

        with tarfile.open(os.path.join(self.root, self.filename), "r:gz") as tar:
            tar.extractall(path=self.root)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data.iloc[idx]
        path = os.path.join(self.root, self.base_folder, sample.filepath)
        target = sample.target - 1  # Targets start at 1 by default, so shift to 0
        img = self.loader(path)

        if self.transform is not None:
            img = self.transform(img)

        return img, target

In [6]:
birds = Cub2011(root=os.path.expanduser("~/torch_datasets/CUB_200_2011"))

Files already downloaded and verified


In [10]:
flowers[100]

(<PIL.Image.Image image mode=RGB size=500x531>, 10)

In [7]:
flowers_and_birds = torch.utils.data.ConcatDataset([flowers, birds])

In [18]:
torchfile.load("/Users/buckhousefamily/Downloads/cvpr2016_flowers/vocab.t7")

{b'onit': 2,
 b'honeycomb': 3,
 b'same': 4,
 b'semi': 5,
 b'bid': 6,
 b'dainty': 7,
 b'graceful': 8,
 b'leads': 9,
 b'accept': 10,
 b'multitude': 11,
 b'mittlea': 12,
 b'clean': 13,
 b'sheer': 14,
 b'inflorescence': 15,
 b'pollinating': 16,
 b'happens': 17,
 b'knitted': 18,
 b'dusky': 19,
 b'lead': 20,
 b'densley': 21,
 b'oink': 22,
 b'upward': 23,
 b'sheet': 24,
 b'spottednear': 25,
 b'flanked': 26,
 b'roomy': 27,
 b'plums': 28,
 b'tightly': 29,
 b'sprouts': 30,
 b'purprle': 31,
 b'receptacles': 32,
 b'generally': 33,
 b'wich': 34,
 b'meedle': 3399,
 b'petals': 36,
 b'stemmed': 37,
 b'shot': 38,
 b'mean': 39,
 b'knife': 40,
 b'six': 41,
 b'remind': 42,
 b'quills': 43,
 b'sticks': 44,
 b'turns': 45,
 b'florets': 46,
 b'transitions': 47,
 b'fingerlike': 48,
 b'combination': 49,
 b'webbing': 50,
 b'tie': 51,
 b'notch': 52,
 b'handful': 53,
 b'rows': 54,
 b'baseballs': 55,
 b'appear': 56,
 b'enfolding': 57,
 b'postil': 58,
 b'shapd': 59,
 b'usually': 60,
 b'daisies': 61,
 b'roubded': 62,
